# Data Augmentation

In [1]:
import os
import cv2
import glob
import keras
from keras.utils.image_utils import img_to_array, array_to_img, load_img
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#STARE
def data_augmentation():
    img_dir = '../data/filtered-data/ODIR/Pure AMD' #orginal directory
    aug_dir = '../data/filtered-data/ODIR/Pure AMD/Augmented' #augmented directory
    data_path = os.path.join(img_dir,'*g')

    files = glob.glob(data_path) #error @ rgb_img when removing glob.glob dunno y
    data = []
    for f1 in files:
        img = cv2.imread(f1) #read img
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #convert BGR img to RGB (depends)
        data.append(rgb_img) 

        x = img_to_array(rgb_img)  
        x = x.reshape((1,) + x.shape)  

        path, dirs, files = next(os.walk(img_dir))
        file_count = len(files)

        datagen = ImageDataGenerator(horizontal_flip = True)

        i = 0
        for batch in datagen.flow(x, batch_size = 1, save_to_dir = aug_dir, save_prefix = 'ODIR_Augment', save_format = 'jpg'):
            i += 1
            if i == 1:
                break  

# Red Channel Extraction & CLAHE

In [3]:
import numpy as np
import os
from os import listdir

In [64]:
def rce_clahe(img_dir, red_dir):
    if not os.path.exists(red_dir):
        os.mkdir(red_dir)

    datagen = ImageDataGenerator(horizontal_flip = True)

    i = 0
    for images in os.listdir(img_dir): 
        # check if the image ends with png or jpg or jpeg
        if (images.endswith(".png") or images.endswith(".jpg")\
            or images.endswith(".jpeg")):
            img = cv2.imread(img_dir + "/" + images, cv2.IMREAD_UNCHANGED)

            # exclude red channel
            b,g,r = cv2.split(img)

            # apply CLAHE
            clahe = cv2.createCLAHE(clipLimit=5)
            b_clahe_enhanced = clahe.apply(b) + 30 # increase intensity of contrast
            g_clahe_enhanced = clahe.apply(g) + 30
            r_clahe_enhanced = clahe.apply(r)

            # merge enhanced channels
            clahe_enhanced_img = cv2.merge((b_clahe_enhanced, g, r_clahe_enhanced))

            grayscale = cv2.cvtColor(clahe_enhanced_img, cv2.COLOR_BGR2GRAY)

            path = os.path.join(red_dir, "ODIR_CLAHE_" + images.split('.')[0] + ".png")
            cv2.imwrite(path , grayscale)
            i+=1

In [67]:
rce_clahe('../data/preprocessing/filtered-data/ODIR/Pure AMD', '../data/preprocessing/CLAHE/ODIR/Pure AMD')
rce_clahe('../data/preprocessing/filtered-data/ODIR/AMD w others', '../data/preprocessing/CLAHE/ODIR/AMD w others')
rce_clahe('../data/preprocessing/filtered-data/ODIR/Normal', '../data/preprocessing/CLAHE/ODIR/Normal')

In [ ]:
rce_clahe('../data/preprocessing/filtered-data/RFMiD/Pure AMD', '../data/preprocessing/CLAHE/RFMiD/Pure AMD')
rce_clahe('../data/preprocessing/filtered-data/RFMiD/AMD w others', '../data/preprocessing/CLAHE/RFMiD/AMD w others')
rce_clahe('../data/preprocessing/filtered-data/RFMiD/Normal', '../data/preprocessing/CLAHE/RFMiD/Normal')